# Reinforcement Learning course notebook

## Environment simulation

We will start with a simple control environment that you will complete yourselves, then we will move to a more complex open-source aircraft control environment.

### Simple line control environment

We will begin with a very simple control problem consisting in maintaining a moving object along a straight line.
We can control its acceleration as shown in the following picture.
The objective is to keep minimum the distance between the object and the center line as long as possible.
To make the problem a bit interesting, we constrain the norm of the acceleration to be larger than a given value $a_{min}$: $\forall t > 0, \Vert a(t) \Vert_2 \geqslant a_{min}$.

![Line control environment](line_control_diagram.png)

#### Mathematical modeling of the Reinforcement Learning problem

We want to learn to control the object using Reinforcement Learning.
The state of the system must contain the minimal information required to update the physics of the system from the action, i.e. the acceleration of the object.
In this case, we need at least the position and the speed of the object, thus the state will be defined at any time $t$ by $s(t)=(x(t), y(t), v_x(t), v_y(t))$
Moreover, for classical RL algorithms to apply, we need to discretize the time every $\Delta t$ time units.
We can now approximate the physics of the object movement with the following equations, knowing the initial state $s(0)=(x(0), y(0), v_x(0), v_y(0))$:
- $\forall t > 0, v_x(t+\Delta t) = v_x(t) + a_x(t) \cdot \Delta t$ ;
- $\forall t > 0, x(t+\Delta t) = x(t) + v_x(t) \cdot \Delta t$ ;
- $\forall t > 0, v_y(t+\Delta t) = v_y(t) + a_y(t) \cdot \Delta t$ ;
- $\forall t > 0, y(t+\Delta t) = y(t) + v_y(t) \cdot \Delta t$ ;

Since we want to keep minimum the distance between the center line and the object, we will model the reward signal at any time $t$ by $r(t) = e^{-\vert y(t) \vert}$.
By doing so, an RL agent who will try to maximize the cumulated sum of (discounted) rewards will try to keep $y(t)$ as close as possible to $0$ at any time step.
There are two possible ways to enforce the constraint $\Vert a(t) \Vert_2 \geqslant a_{min}$ in RL:
- either by ensuring that the algorithm will only select such actions ;
- or by associating a very large penalty (i.e. negative reward) to transitions labelled with such actions.

#### Implementation in a Gym environment

[OpenAI Gym](https://gym.openai.com/) is a popular Python software library to model RL environments in a standard way which can be exploited RL algorithm libraries like [RLlib](https://www.ray.io/rllib) or [Stable Baselines](https://github.com/DLR-RM/stable-baselines3).
OpenAI Gym - or Gym in short - provides well-known environment implementations like CartPole, but we can also implement our own environment by following their standards, which will allow us to solve our environment using well-implemented and efficient RL algorithms from the aforementioned libraries.
All we have to do is to implement a domain class with the following methods:
```python
class MyEnvironement:
    def __init__(self):
        # Declare your variables here, including the environment's state.
        # Declare also the action and observation (i.e. state) spaces: the action space is used by the algorithm
        # to select actions while the observation space is used by Deep RL algorithms to properly initialize
        # the observation (i.e. state) layer of the tensors.
        pass

    def reset(self):
        # Initialize and return the initial state of the environment
        pass

    def step(self, action):
        # Perform one simulation step of the environment, i.e. compute the state resulting from applying the given action in the current state.
        # Don't forget to update the environment's state so that the next call to the step method will reason about the updated state.
        # Must return a tuple (state, reward, done, info) where done is true if the episode should stop now and info is a dictionary that can be left empty.
        pass

    def render(self, mode="human"):
        # If you want to render something at each simulation step (e.g. an image, some text, etc.)
        pass
```

#### It's your turn!
Please fill in the missing lines in the definition below of the Gym environment which implements "simple line control" problem.

In [ ]:
import gym
import numpy as np
import pygame
from pygame import gfxdraw
from math import sqrt, exp, fabs


HORIZON = 500
ACCELERATION_MIN = 0.5
PENALTY = -1000.


class SimpleLineControlGymEnv(gym.Env):
    """This class mimics an OpenAI Gym environment"""
    
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 50}

    def __init__(self, env_config=None):
        """Initialize GymDomain.
        # Parameters
        gym_env: The Gym environment (gym.env) to wrap.
        """
        inf = np.finfo(np.float32).max
        self.action_space = gym.spaces.Box(
            np.array([-1.0, -1.0]), np.array([1.0, 1.0]), dtype=np.float32
        )
        self.observation_space = gym.spaces.Box(
            np.array([-inf, -inf, -inf, -inf]),
            np.array([inf, inf, inf, inf]),
            dtype=np.float32,
        )
        self._delta_t = 0.001
        self._init_pos_x = 0.0
        self._init_pos_y = 0.5
        self._init_speed_x = 10.0
        self._init_speed_y = 1.0
        self._pos_x = None
        self._pos_y = None
        self._speed_x = None
        self._speed_y = None
        self._path = []
        
        self.screen = None
        self.clock = None
        self.isopen = True

    def get_state(self):
        return np.array(
            [self._pos_x, self._pos_y, self._speed_x, self._speed_y], dtype=np.float32
        )

    def set_state(self, state):
        self._pos_x = state[0]
        self._pos_y = state[1]
        self._speed_x = state[2]
        self._speed_y = state[3]

    def reset(self):
        self._pos_x = self._init_pos_x
        self._pos_y = self._init_pos_y
        self._speed_x = self._init_speed_x
        self._speed_y = self._init_speed_y
        self._path = []
        return np.array(
            [self._pos_x, self._pos_y, self._speed_x, self._speed_y], dtype=np.float32
        )

    def step(self, action):
        ### WRITE YOUR CODE HERE
        # If you get stuck, uncomment the line in the next cell to load a solution.     
        self._path.append((self._pos_x, self._pos_y))
        return obs, reward, done, {}

    def render(self, mode="human"):
        screen_width = 600
        screen_height = 400

        if self.screen is None:
            pygame.init()
            pygame.display.init()
            self.screen = pygame.display.set_mode((screen_width, screen_height))
        if self.clock is None:
            self.clock = pygame.time.Clock()
        
        self.surf = pygame.Surface((screen_width, screen_height))
        self.surf.fill((255, 255, 255))
        self.track = gfxdraw.hline(
            self.surf,
            0,
            screen_width,
            int(screen_height / 2),
            (0, 0, 255)
        )

        if len(self._path) > 1:
            for p in range(len(self._path) - 1):
                gfxdraw.line(
                    self.surf,
                    int(self._path[p][0] * 100),
                    int(screen_height / 2 + self._path[p][1] * 100),
                    int(self._path[p+1][0] * 100),
                    int(screen_height / 2 + self._path[p+1][1] * 100),
                    (255, 0, 0)
                )

        self.surf = pygame.transform.flip(self.surf, False, True)
        self.screen.blit(self.surf, (0, 0))
        if mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()

        if mode == "rgb_array":
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )
        else:
            return self.isopen

    def close(self):
        if self.screen is not None:
            pygame.display.quit()
            pygame.quit()
            self.isopen = False


In [ ]:
# %load solutions/line_control.py

In [ ]:
import matplotlib.pyplot as plt
import gym
import random
from IPython import display
%matplotlib inline

env = SimpleLineControlGymEnv()
obs = env.reset()

for i in range(50):
    plt.imshow(env.render(mode='rgb_array'))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    obs, reward, done, info = env.step(
        tuple([random.uniform(-1., 1.), random.uniform(-1., 1.)])
    )
    
env.close()

In [ ]:
import sys
sys.modules[__name__]

try:
  gym.envs.register(
      id='simple_line_control_env-v0',
      entry_point='__main__:SimpleLineControlGymEnv',
      max_episode_steps=320
  )
except:
    pass

In [ ]:
import ray
from ray.tune.registry import register_env

# Import the RL algorithm (Trainer) we would like to use.
from ray.rllib.agents.ppo import PPOTrainer
# from ray.rllib.agents.sac import SACTrainer

register_env("simple_line_control_env-v0",
             lambda cfg: SimpleLineControlGymEnv(cfg))

# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "simple_line_control_env-v0",
    "env_config": {},
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 2,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    }
}

ray.init(ignore_reinit_error=True)

# Create our RLlib Trainer.
trainer = PPOTrainer(config=config)


# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for _ in range(100):
    trainer.train()

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
trainer.evaluate()

ray.shutdown()

### Aircraft taxiing control environment

Now we investigate a more complex control problem consisting in controlling a flying aircraft.
Based on the Gym environments from the [gym-jsbsim](https://github.com/galleon/gym-jsbsim) library which simulate aircraft physics,
we will try to learn to follow a certain heading and altitude so that every 150 sec a new target heading and altitude are set.
The environment is explained [here](https://github.com/galleon/gym-jsbsim/blob/master/README.md#heading-and-altitude-task).

In [ ]:
import gym
import gym_jsbsim

env = gym.make("GymJsbsim-HeadingAltitudeControlTask-v0")
env.reset()
done = False

while not done:
   action = env.action_space.sample()
   state, reward, done, _ = env.step(action)
   print('state: {}'.format(state))